In [1]:
# Traffic data is in the '.dbf' format, so I got a package to read it
from dbfread import DBF

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import json
import seaborn as sns
import re
import statistics

%matplotlib inline 

In [2]:
dbf = DBF('../data/traffic/TN Traffic Dataset/TrfcHist.dbf')
og_df = pd.DataFrame(iter(dbf))


In [3]:
og_df

,STATION_ID,STN_NUMBER,COUNTY,LOCATION,RTE_NUMBER,AADT_2018,AADT_2017,AADT_2016,AADT_2015,AADT_2014,...,AADT_1990,AADT_1989,AADT_1988,AADT_1987,AADT_1986,AADT_1985,AADT_1984,AADT_1983,X,Y
0,001000005,000005,Anderson,NEAR SCOTT CO LINE,SR116,268.0,181.0,315.0,309.0,255.0,...,840.0,1150.0,1200.0,1233.0,1121.0,712.0,635.0,1496.0,-84.416817,36.126099
1,001000007,000007,Anderson,S OF LAKE CITY & N OF CLINTON,SR116,1765.0,1709.0,1841.0,1747.0,1767.0,...,2174.0,1995.0,2208.0,2532.0,1914.0,1939.0,2170.0,2035.0,-84.181373,36.191066
2,001000008,000008,Anderson,SW LAKE CITY,SR116,3240.0,3260.0,3421.0,3291.0,3238.0,...,4026.0,3834.0,4209.0,4919.0,4252.0,3622.0,3683.0,3334.0,-84.162139,36.218138
3,001000010,000010,Anderson,S OF I-75 NEAR CAMPBELL CO LINE,SR071,1936.0,1926.0,2174.0,1851.0,2202.0,...,2056.0,1770.0,1932.0,1598.0,2014.0,1546.0,1442.0,1366.0,-84.155918,36.228584
4,001000011,000011,Anderson,S OF LAKE CITY,SR009,8956.0,7940.0,8564.0,8381.0,8797.0,...,10424.0,9000.0,9282.0,8654.0,10284.0,8684.0,8994.0,8003.0,-84.152704,36.214983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12452,189000234,000234,Wilson,PROVIDENCE PKWY. - E. OF SR-171,06103,12831.0,14016.0,13877.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-86.512780,36.163983
12453,189000235,000235,Wilson,PROVIDENCE TR. - S. OF BELINDA PKWY,06104,4562.0,4032.0,4052.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-86.503559,36.167042
12454,189000236,000236,Wilson,GOLDEN BEAR GATEWAY - S. OF SR-24,06105,4649.0,3539.0,3504.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-86.491442,36.231025
12455,189000237,000237,Wilson,BECKWITH RD - N. OF I-40,01368,1515.0,1500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-86.456851,36.176016


In [4]:
dbf = DBF('../data/traffic/TN Traffic by Year/TrfcHistYR.dbf')
og_year_df = pd.DataFrame(iter(dbf))


In [5]:
og_year_df

,STATION_ID,STN_NUMBER,COUNTY,LOCATION,YEAR,ADJAVGDLY,RTE_NUMBER,X,Y
0,001000005,000005,Anderson,NEAR SCOTT CO LINE,2018,268.0,SR116,-84.416817,36.126099
1,001000005,000005,Anderson,NEAR SCOTT CO LINE,2017,181.0,SR116,-84.416817,36.126099
2,001000005,000005,Anderson,NEAR SCOTT CO LINE,2016,315.0,SR116,-84.416817,36.126099
3,001000005,000005,Anderson,NEAR SCOTT CO LINE,2015,309.0,SR116,-84.416817,36.126099
4,001000005,000005,Anderson,NEAR SCOTT CO LINE,2014,255.0,SR116,-84.416817,36.126099
...,...,...,...,...,...,...,...,...,...
376808,189000990,000990,Wilson,W. OF SR-109 (ATR 34),1989,37149.0,I0040,-86.458054,36.178037
376809,189000990,000990,Wilson,W. OF SR-109 (ATR 34),1988,35715.0,I0040,-86.458054,36.178037
376810,189000990,000990,Wilson,W. OF SR-109 (ATR 34),1987,33590.0,I0040,-86.458054,36.178037
376811,189000990,000990,Wilson,W. OF SR-109 (ATR 34),1986,29700.0,I0040,-86.458054,36.178037


In [6]:
davidson_year = og_year_df[og_year_df['COUNTY'] == 'Davidson']
davidson = og_df[og_df['COUNTY'] == 'Davidson']

In [7]:
# Filtering it to davidson county
davidson_year

,STATION_ID,STN_NUMBER,COUNTY,LOCATION,YEAR,ADJAVGDLY,RTE_NUMBER,X,Y
61822,037000001,000001,Davidson,NEAR ROBERTSON CO. LINE,2018,20589.0,SR011,-86.757874,36.387809
61823,037000001,000001,Davidson,NEAR ROBERTSON CO. LINE,2017,20210.0,SR011,-86.757874,36.387809
61824,037000001,000001,Davidson,NEAR ROBERTSON CO. LINE,2016,19210.0,SR011,-86.757874,36.387809
61825,037000001,000001,Davidson,NEAR ROBERTSON CO. LINE,2015,17520.0,SR011,-86.757874,36.387809
61826,037000001,000001,Davidson,NEAR ROBERTSON CO. LINE,2014,17776.0,SR011,-86.757874,36.387809
...,...,...,...,...,...,...,...,...,...
77737,037000994,000994,Davidson,ELLINGTON PKWY (ATR 4) S. OF EAST TRINITY LN.,2017,53737.0,SR006,-86.751053,36.201194
77738,037000994,000994,Davidson,ELLINGTON PKWY (ATR 4) S. OF EAST TRINITY LN.,2016,48457.0,SR006,-86.751053,36.201194
77739,037000994,000994,Davidson,ELLINGTON PKWY (ATR 4) S. OF EAST TRINITY LN.,2015,47033.0,SR006,-86.751053,36.201194
77740,037000994,000994,Davidson,ELLINGTON PKWY (ATR 4) S. OF EAST TRINITY LN.,2014,43271.0,SR006,-86.751053,36.201194


In [8]:
davidson

,STATION_ID,STN_NUMBER,COUNTY,LOCATION,RTE_NUMBER,AADT_2018,AADT_2017,AADT_2016,AADT_2015,AADT_2014,...,AADT_1990,AADT_1989,AADT_1988,AADT_1987,AADT_1986,AADT_1985,AADT_1984,AADT_1983,X,Y
1965,037000001,000001,Davidson,NEAR ROBERTSON CO. LINE,SR011,20589.0,20210.0,19210.0,17520.0,17776.0,...,14839.0,12600.0,12500.0,12229.0,10972.0,8540.0,0.0,0.0,-86.757874,36.387809
1966,037000002,000002,Davidson,N. OF GOODLETTSVILLE,SR011,21275.0,21769.0,19787.0,19533.0,18231.0,...,16360.0,14150.0,12726.0,13500.0,13307.0,11636.0,0.0,0.0,-86.725259,36.358267
1967,037000004,000004,Davidson,OLD SPRINGFIELD PK. - N. OF GOODLETTSVILLE,04927,2724.0,2762.0,2538.0,2387.0,2013.0,...,2714.0,3829.0,2781.0,2915.0,2800.0,2744.0,0.0,0.0,-86.725828,36.346429
1968,037000008,000008,Davidson,N OF JOELTON,SR065,11550.0,13072.0,11535.0,11306.0,11241.0,...,8108.0,6504.0,6552.0,5709.0,4919.0,4605.0,0.0,0.0,-86.871530,36.339774
1969,037000011,000011,Davidson,NEAR CHEATHAM CO LINE,SR112,8597.0,8417.0,7529.0,6553.0,6362.0,...,4858.0,4320.0,4370.0,4281.0,4089.0,3714.0,0.0,0.0,-86.904144,36.321489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2500,037000644,000644,Davidson,OTTER CREEK RD - W. OF SR-6,05964,4500.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-86.778271,36.065493
2501,037000990,000990,Davidson,N OF GOODLETTSVILLE (ATR 20) 301/20),SR011,18674.0,18686.0,17375.0,19464.0,18332.0,...,18694.0,21378.0,20717.0,20161.0,16520.0,15345.0,0.0,0.0,-86.716653,36.332246
2502,037000991,000991,Davidson,S OF BRILEY PKWY (ATR 33),I0024,190529.0,186329.0,176880.0,169896.0,172117.0,...,81953.0,98498.0,96740.0,0.0,0.0,0.0,0.0,0.0,-86.712769,36.101545
2503,037000993,000993,Davidson,NEAR MILL CREEK (ATR 555),SR001,23442.0,24301.0,22382.0,21452.0,20934.0,...,28169.0,37800.0,37305.0,31417.0,38020.0,42279.0,0.0,0.0,-86.705665,36.122403
